# 零樣本音訊分類 Zero-Shot Audio Classification

Zero-Shot Audio Classification 是指一種音訊分類的方法，其中模型在訓練過程中沒有接觸過特定類別的音訊樣本，但能夠在推論階段有效地將其分類到先前未見過的類別中。這種方法的核心思想是利用模型在訓練期間學習到的通用特徵來進行推論，而不是僅僅依賴於訓練集中的特定類別的示例。

在 Zero-Shot Audio Classification 中，通常會使用預訓練的音訊處理模型，例如基於深度學習的模型，如聲學模型或深度神經網絡。這些模型通常在大規模的音訊數據集上進行預訓練，從而學習到了音訊的通用特徵。然後，這些模型可以在推論階段對新的音訊樣本進行分類，即使這些樣本屬於模型在訓練期間未見過的類別。

Zero-Shot Audio Classification 的應用範圍包括音訊標籤預測、音訊事件檢測等任務，尤其在數據稀缺或新類別出現的情況下，具有重要的應用價值。


- 如果您在自己的電腦上執行此程式碼，請安裝以下內容:
```
    !pip install transformers
    !pip install datasets
    !pip install soundfile
    !pip install librosa
```
`librosa` 函式庫可能需要安裝 [ffmpeg](https://www.ffmpeg.org/download.html)。
- [librosa](https://pypi.org/project/librosa/) 上的此頁面提供了 ffmpeg 的安裝說明。


- 這是一些抑制警告訊息的程式碼。

In [ ]:
from transformers.utils import logging
logging.set_verbosity_error()

### 準備音訊資料集

[ESC50](https://huggingface.co/datasets/ashraq/esc50) 環境聲音分類資料集。第 23 屆 ACM 多媒體年度會議論文集，澳洲布里斯班，2015 年。

K. J. Piczak. ESC: Dataset for Environmental Sound Classification. Proceedings of the 23rd Annual ACM Conference on Multimedia, Brisbane, Australia, 2015.

ashraq/esc50

In [ ]:
from datasets import load_dataset, load_from_disk

# 此資料集是5秒不同聲音的集合
dataset = load_dataset("ashraq/esc50", split="train[0:10]")

len(dataset)

audio_sample = dataset[0]
audio_sample

#播放聲音
from IPython.display import Audio as IPythonAudio
IPythonAudio(audio_sample["audio"]['array'], rate=audio_sample["audio"]['sampling_rate'])

### 使用 🤗 Transformers 庫建立「音訊分類」管道

laion/clap-htsat-unfused

In [ ]:
from transformers import pipeline
zero_shot_classifier = pipeline('zero-shot-audio-classification',
                                model = 'laion/clap-htsat-unfused')

### Transformer 模型的取樣率 Sampling Rate
- 用 Whisper 模型（經過訓練預測音訊檔案的頻率為 16,000 Hz）,1 秒的高解析度音訊 (192,000 Hz) 會有多久時間？


In [ ]:
192000 / 16000

- 1 秒的高解析度音訊對於模型來說就像是 12 秒的音訊。

- 5秒的音頻怎麼樣？

In [ ]:
(5*192000)/16000

- 5 秒的高解析度音訊對於模型來說就像 60 秒的音訊一樣。

model.feature_extractor.sampling_rate

audio_sample["audio"]['sampling_rate']

* 為輸入和模型設定正確的取樣率。

In [ ]:
import datasets import audio
dataset = dataset.cast_column('audio', Audio(sampling_rate = 48000))
autio_sample = dataset[0]
audio_sample

### Transformer 模型的取樣率 Sampling Rate
- 用 Whisper 模型（經過訓練預測音訊檔案的頻率為 16,000 Hz）,1 秒的高解析度音訊 (192,000 Hz) 會有多久時間？


In [ ]:
# 狗的聲音, 吸塵器的聲音
candidate_labels = ["Sound of a dog",
                    "Sound of vacuum cleaner"]
res1 = model(audio_sample["audio"]['array'], candidate_labels=candidate_labels)
for r in res1:
  print(f"{r['label']} 相似分數:{r['score']:.4f}")

candidate_labels = ["Sound of a child crying",
                    "Sound of vacuum cleaner",
                    "Sound of a bird singing",
                    "Sound of an airplane"]
audio_sample= dataset[7]
res2 = model(audio_sample["audio"]['array'], candidate_labels=candidate_labels)
for r in res2:
  print(f"{r['label']} 相似分數:{r['score']:.4f}")
IPythonAudio(audio_sample["audio"]['array'], rate=audio_sample["audio"]['sampling_rate'])

## gradio

In [ ]:
def guessSound(audio_sample, candidate_labels):
  res = model(audio_sample["audio"]['array'], candidate_labels=candidate_labels)
  text = ''
  for r in res:
    text += f"{r['label']} 相似分數:{r['score']:.4f}\n"
  return text
candidate_labels = ["Sound of a child crying",
                    "Sound of vacuum cleaner",
                    "Sound of a bird singing",
                    "Sound of an airplane",
                    "Sound of thunderstorm",
                    "Sound of can_opening",
                    "Sound of a dog"]
print(guessSound(dataset[0], candidate_labels))
IPythonAudio(dataset[8]["audio"]['array'], rate=dataset[8]["audio"]['sampling_rate'])